# Eye Diagrams
<p style="font-size:18px">
This notebook demonstrates how amplitude noise, bandwidth limitations, and timing jitter lead to errors in determining the value of a transmitted bit, and how an eye diagram provides a visual illustration of the contributions to errors.

## Amplitude Noise, Q-Factor, and Bit Error Rate
<p style="font-size:18px">
One of the most important contributions to errors in the detection of optical signals is thermal noise in the optical receiver. Thermal noise arises from fluctuations in the current through a resitive load. It has a Gaussian distribution that is characterized by the standard deviation for the noise current, given by the expression
<p style="font-size:18px">
$${\sigma _T} = \sqrt {\frac{{4{k_B}T}}{{{R_L}}}\Delta f}, $$
where $k_B$ is Boltzman's constant, T is the temperature, $R_L$ is the resistance of the load, and $\Delta f$ is the bandwidth of the receiver. Current variation leads to a variation in the voltage across the resisttor and and to a variation in the amplitude of the measured signal.
<p style="font-size:18px">
Thermal noise constibutes equally to variation in the amplitude for transmitted "ones" and "zeros". More generally there are additional noise sources that contibute in different amounts to ones and zeros (shot noise is an example), and that are characterized by two different standard deviations ${\sigma _1}$ and ${\sigma _0}$.
<p style="font-size:18px">
A quantity called the Q-Factor is the generalization for signal-to-noise-ratio that is used when the amount of noise depends on signal level. The Q-Factor is given by the expression
<p style="font-size:18px">
$$Q \equiv \frac{{{I_1} - {I_0}}}{{{\sigma _0} + {\sigma _1}}},$$
<p style="font-size:18px">
where $I_1$ and $I_0$ are the average current generated by a transmitted one and zero, respectively. The bit error rate (BER, the fraction of bits incorrectly detected) is related to Q-Factor  by the expression
<p style="font-size:18px">
$$BER = \frac{1}{2}erfc\left( {\frac{Q}{{\sqrt 2 }}} \right),$$
where erfc is the complementary error function.
### Exploration
<p style="font-size:18px">
Explore how combinations of signal ampliture and signal noise contibute to the Q-Factor and bit-error rate with the interactive graphs below. The graph on the left shows a signal that represents the bit sequence 0,1,0. Just below, the graph on the right shows the distribution of the signal for this sequence. A transmitted bit is determined to be a one if a measurment is above the voltage threshold, and a zero if a measurement is below the threshold. Signals are measured at the middle of a bit period. As is particulary apparent from the graph on the rigth, there is always a non-zero probability that a signal will be on the wrong side of the threshold level during a measurement, resulting in an incorrect translation of a signal into a bit.
<p style="font-size:18px">
Suggestion: Adjust the average received power and noise levels to achieve a bit error rate of 1 x $10^{-12}$. Note the corresponding Q-Factor.
<p style="font-size:18px">
Suggestion: Adjust the sliders so that the signal level for a one occasionally falls below the threshold level, and examine the corresponding Q-Factor and bit error rate.

In [1]:
from ipywidgets import interact, FloatSlider, IntSlider, Button, HBox
import numpy as np
from numpy import fft
from scipy import special
from scipy.interpolate import CubicSpline
from bokeh.io import push_notebook
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, Range1d, LinearAxis, Title, Label, Span
from bokeh.layouts import column, row
from bokeh.models.glyphs import Line
output_notebook()

Loading BokehJS ...

In [2]:
kB = 1.381e-23 # Boltzmann Constant
T = 300 # Temperature
R = 0.5 # Responsivity
RL = 50 # Load Resistor, Ohms
G = 10 # Post Load Amplifier Gain
bandwidth = 2.5 # Receiver Bandwdith in GHz
sigmaT = np.sqrt(4*kB*T/RL*bandwidth*1e9)
Pave = 5; sigma0 = sigmaT; sigma1 = sigmaT # voltage in mV
Q = (2*Pave/1000000*R*RL*G/(sigma0+sigma1+1e-6)) # The Q-Factor that gives BER
BER = 0.5*special.erfc((Pave/1000000*2*R*RL*G)/(sigma0*G+sigma1*G)/np.sqrt(2)) # Bit Error Rate

v1 = np.random.normal(0, sigma0*G, 1000)*1e6
v2 = np.random.normal(Pave/1000000*2*R*RL*G, sigma1, 1000)*1e6
v3 = np.random.normal(0, sigma0, 1000)*1e6
v = np.concatenate([v1,v2,v3])
t=np.linspace(0,1200,3000) # time in picoseconds
zeros=np.zeros(3000)
Threshold=zeros+Pave/1000000*R*RL*G
p = figure(plot_width=400, plot_height=400, background_fill_color="lightgray", title='Signal with Amplitude Noise')
p.title.text_font_style='normal'; p.title.text_font_size='12pt'
p.outline_line_width = 1; p.outline_line_color = "black"; p.min_border_top = 10
l=p.line(t,v,line_color='#2222aa')
lt=p.line(t,Threshold,line_dash='dashed',line_color='red')
p.x_range = Range1d(0, 1200)
# p.y_range = Range1d(-0.0, 2.0)
p.xaxis.axis_label="Time (psec)"; p.xaxis.major_label_text_font_size = "12pt"
p.xaxis.axis_label_text_font_style = "normal"; p.xaxis.axis_label_text_font_size = "12pt"
p.yaxis.axis_label="Voltage (mV)"; p.yaxis.major_label_text_font_size = "12pt"
p.yaxis.axis_label_text_font_style = "normal"; p.yaxis.axis_label_text_font_size = "12pt";
label=Label(x=800, y=5.5, text='Q='+str(Q), text_color='darkblue', background_fill_color='lightgray')
label1=Label(x=40, y=Pave/1000000*R*RL*G, text='V Threshold', text_color='darkblue', background_fill_color='lightgray')
p.add_layout(label); p.add_layout(label1)

p1 = figure(plot_width=400, plot_height=400, background_fill_color="lightgray", title='Noise Distribution',
           y_axis_type="log")
p1.title.text_font_style='normal'; p1.title.text_font_size='12pt'
p1.outline_line_width = 1; p1.outline_line_color = "black"
p1.y_range = Range1d(1e-15, 1e2)
vline = Span(location=0.2,dimension='height', line_color='red', line_dash='dashed')
p1.add_layout(vline)

hist1, edges1 = np.histogram(v1, range=(-1,6), density=True, bins=100)
hist2, edges2 = np.histogram(v2, range=(-1,6), density=True, bins=100)
q1 = p1.quad(top=hist1, bottom=0, left=edges1[:-1], right=edges1[1:],
        fill_color="#036564", line_color="#033649")
q2 = p1.quad(top=hist2, bottom=0, left=edges2[:-1], right=edges2[1:],
        fill_color="#036564", line_color="#033649")
ivalue=np.linspace(-0.5,5.5,1000)
ql1 = p1.line(ivalue, (1/(sigma0*(2*np.pi)**0.5)*np.exp(-(ivalue-0)**2/sigma0**2)), line_width=2, line_color='darkorange')
ql2 = p1.line(ivalue, (1/(sigma1*(2*np.pi)**0.5)*np.exp(-(ivalue-Pave*2*R*RL*G)**2/sigma1**2)), line_width=2, line_color='darkorange')
p1.xaxis.axis_label="Voltage (mV)"; p1.xaxis.major_label_text_font_size = "12pt"
p1.xaxis.axis_label_text_font_style = "normal"; p1.xaxis.axis_label_text_font_size = "12pt"
p1.yaxis.axis_label="Probabilty"; p1.yaxis.major_label_text_font_size = "12pt"
p1.yaxis.axis_label_text_font_style = "normal"; p1.yaxis.axis_label_text_font_size = "12pt"
label2=Label(x=3.4,  y=2.3, text='BER='+ str(BER),
                                             text_color='darkblue', background_fill_color='lightgray')
label3=Label(x=Pave*R*RL*G, y=2.1, text='V Threshold', text_color='darkblue', background_fill_color='lightgray')
p1.add_layout(label2); p1.add_layout(label3) 


label2=Label(x=3.2,  y=5.0, text='BER='+str(BER),
                text_color='darkblue', background_fill_color='lightgray')
label3=Label(x=Pave*R*RL*G, y=0.01, text='V Threshold', text_color='darkblue', background_fill_color='lightgray')
p1.add_layout(label2); p1.add_layout(label3) 

Pave_slider=FloatSlider(min=0, max=2, step=0.1, value = 1.0, description='Pave (\u03BCW)', continuous_update=True)
# sigma0_slider=FloatSlider(min=0.0, max=0.8, step=0.05, value=0.2, description='sigma-0 (mV)', continuous_update=True)
# sigma1_slider=FloatSlider(min=0.0, max=0.8, step=0.05, value=0.2, description='sigma-1 (mV)', continuous_update=True)

def replot(Pave):
    Pave = Pave # Divide Pave by 1000 to give voltages in mV
    v1 = np.random.normal(0, sigma0*G, 1000)
    v2 = np.random.normal(Pave/1000000*2*R*RL*G, sigma1*G, 1000)
    v3 = np.random.normal(0, sigma0*G, 1000)
    v = np.concatenate([v1,v2,v3])
    lt.data_source.data['y']=zeros+Pave/1000000*R*RL*G
    l.data_source.data['y']=v
    q1.data_source.data['top']=hist1
    q2.data_source.data['top']=hist2
    ql1.data_source.data['y']=1/(sigma0*(2*np.pi)**0.5+1e-9)*np.exp(-(ivalue-0)**2/(sigma0**2+1e-9))
    ql2.data_source.data['y']=1/(sigma1*(2*np.pi)**0.5+1e-9)*np.exp(-(ivalue-Pave*2*R*RL*G)**2/(sigma1**2+1e-9))
    vline.location=Pave*R*RL*G
    Q = (2*Pave/1000000*R*RL*G)/(sigma0*G+sigma1*G+1e-6); label.text='Q-Factor='+str(Q)[:4]
    label1.y=Pave*R*RL*G
    BER = 0.5*special.erfc(Q/np.sqrt(2))
    label2.text='BER='+'{:.2e}'.format(BER)
    label3.x=Pave*R*RL*G
    push_notebook(handle=fig_handle)

fig_handle = show(row(p,p1), notebook_handle=True)
interact(replot, Pave=Pave_slider);

C:\Users\Alan\anaconda3\envs\workingenv\lib\site-packages\numpy\lib\histograms.py:905: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges


interactive(children=(FloatSlider(value=1.0, description='Pave (μW)', max=2.0), Output()), _dom_classes=('widg…

## Finite Bandwidth and Signal Jitter
<p style="font-size:18px">
The bandwidth of optical receivers is typically limited to the bit rate. For example, the bandwidth for an optical receiver for detecting a bit stream at 2.5 gigabits per second will be on the order of 2.5 GHz. A finite bandwidth has the effect of reducing thermal noise and error rates, as can be seen from the equation for $\sigma_T$ given above. On the other hand, finite bandwidth "rounds" signals and reduces the "flat" regions in the middle of bit periods where measurments take place. When combined with signal jitter (a random error in the timing of the transitions between bit periods), this can lead to increased error rates.
### Exploration
<p style="font-size:18px">
Explore how combinations of bandwidth and timing jitter distort signals. Just below, the cureve on the left shows the frequency components for a square signal with an amplitude of 2.5 mV and added thermal noise contribution $\sigma _T=1.52 x 10^{-5} amp$. The graph also shows the filtering reponse of an optical receiver with a 3dB bandwdith set by a slider. The graph on the right shows the filtered/bandwidth-limited signal.

<p style="font-size:18px">
Suggesion: Find a combination of bandwidth and jitter for which you can see the signal fall below (or almost fall below) the voltage threshold at the center of the bit period for the "1" (i.e. at 600 picoseconds).

In [3]:
R = 0.5 # Responsivity
RL = 50 # Load Resistor, Ohms
G = 10 # Post Load Amplifier Gain
bandwidth = 5.0 # The 3 dB bandwidth of the receiver
Pave = 5/1000; sigma0 = 7.6; sigma1 = 7.6 # Divide Pave by 1000 to give voltage in mV
Q = (2*Pave*R*RL*G/(sigma0+sigma1)) # The Q-Factor that gives BER
BER = 0.5*special.erfc((Pave*2*R*RL*G)/(sigma0+sigma1)/np.sqrt(2)) # Bit Error Rate

v1 = np.random.normal(0, sigma0, 1000)
v2 = np.random.normal(Pave*2*R*RL*G, sigma1, 1000)
v3 = np.random.normal(0, sigma0, 1000)
v = np.concatenate([v1,v2,v3])
t=np.linspace(0,1200,3000) # time in picoseconds
Threshold2=zeros+Pave*R*RL*G

sig_fft= np.fft.rfft(v)
sig_fft_norm = np.absolute(sig_fft)/np.amax(np.absolute(sig_fft))
fft_freq = np.linspace(0,1500/1.200,1501)
filter=1/(fft_freq**2/bandwidth**2+1)+1e-2

p2 = figure(plot_width=400, plot_height=400,y_axis_type="log", background_fill_color="lightgray")
p2.outline_line_width = 1; p2.outline_line_color = "black"; p2.min_border_top = 10
lfft = p2.line(fft_freq,sig_fft_norm,line_color='#2222aa', legend_label = 'Frequency Components')
p2.legend.background_fill_alpha = 0.7
lf = p2.line(fft_freq,filter, line_color='red', legend_label = 'Filter Transmission')
p2.x_range = Range1d(0, 80); p2.y_range = Range1d(0.001, 1)
p2.xaxis.axis_label="Frequency (GHz)"; p2.xaxis.major_label_text_font_size = "12pt"
p2.xaxis.axis_label_text_font_style = "normal"; p2.xaxis.axis_label_text_font_size = "12pt"
p2.yaxis.axis_label="Normalized Amplitude"; p2.yaxis.major_label_text_font_size = "12pt"
p2.yaxis.axis_label_text_font_style = "normal"; p2.yaxis.axis_label_text_font_size = "12pt"

sig_fft_f = sig_fft*filter
inv_sig_fft=np.fft.irfft(sig_fft_f)

p3 = figure(plot_width=400, plot_height=400, background_fill_color="lightgray",
            title="Bandwidth Limited Signal")
p3.title.text_font_style='normal'; p3.title.text_font_size='12pt'
p3.outline_line_width = 1; p3.outline_line_color = "black"
p3.x_range = Range1d(0, 1200); p3.y_range = Range1d(-0.75, 3.25)
lbl = p3.line(t,inv_sig_fft,line_color='#2222aa', line_alpha=0.2)
lb2 = p3.line(t,inv_sig_fft,line_color='#2222aa')
lt2 = p3.line(t,Threshold2,line_dash='dashed',line_color='red')
p3.xaxis.axis_label="Time (psec)"; p3.xaxis.major_label_text_font_size = "12pt"
p3.xaxis.axis_label_text_font_style = "normal"; p3.xaxis.axis_label_text_font_size = "12pt"
p3.yaxis.axis_label="Amplitude"; p3.yaxis.major_label_text_font_size = "12pt"
p3.yaxis.axis_label_text_font_style = "normal"; p3.yaxis.axis_label_text_font_size = "12pt"
label4=Label(x=20, y=Pave*R*RL*G, text='V Threshold')
p3.add_layout(label4)

style = {'description_width': 'initial'}
bandwidth_slider=FloatSlider(min=0.5, max=15.0, step=0.05, value = 5.0, description='3dB BW (GHz)',
                             continuous_update=True, style=style)
jitter_slider=FloatSlider(min=-140, max=140, step=34, value = 0, description='Jitter (psec)',
                             continuous_update=True, style=style)

def replot2(bandwidth, jitter):
    filter=1/(fft_freq**2/bandwidth**2+1)+1e-2
    lf.data_source.data['y']=filter
    sig_fft_f = sig_fft*filter
    inv_sig_fft=np.fft.irfft(sig_fft_f)
    lbl.data_source.data['y']=inv_sig_fft
    lb2.data_source.data['x']=t-jitter
    lb2.data_source.data['y']=inv_sig_fft
    lt2.data_source.data['y']=zeros+np.average(inv_sig_fft[:2000])
    label4.y=np.average(inv_sig_fft[:2000])
    push_notebook(handle=fig_handle2)

fig_handle2 = show(row(p2,p3), notebook_handle=True)
interact(replot2, bandwidth=bandwidth_slider, jitter=jitter_slider);

interactive(children=(FloatSlider(value=5.0, description='3dB BW (GHz)', max=15.0, min=0.5, step=0.05, style=S…

## Building an Eye Diagram
<p style="font-size:18px">
A measurment of bit error rate provides a single, quantitative value that provides a valuable, overall assessment of the peformance of an optical link. An eye diagram is a visual tool that can be used to quickly produce a qualtitative assessment of the various contributions to the bit error rate, including themal noise, finite bandwidth, and timing jitter. An eye diagram is created by overlapping multiple signal traces, each carrying a separate random bit pattern. The traces are typically two bit periods long, with half a bit period followed by a full bit period, followed by half a bit period.

### Exploration
<p style="font-size:18px">
The graph just below shows how an eye diagram is built up by drawing multiple segments of bit stream. Click on the "Trace" button to draw a signal curve on the graph. The random bit pattern will be one of eight - 000. 001, 010, 011, 100, 101, 110, or 111. Up to 21 curves can be drawn on the graph. The "Reset" button clears the graph.

<p style="font-size:18px">
Suggestion: Find the minimum number of signal traces needed to create an eye diagram that shows both the vertical and horizontal undertainty in the signal values.

In [4]:
import ipywidgets as widgets
from IPython.display import display
trace_button = Button(description="Trace")
reset_button = Button(description="Reset")
trace_button.style.button_color = 'lightgreen'; reset_button.style.button_color = 'lightgreen'

display(HBox([trace_button, reset_button]))

R = 0.5 # Responsivity
RL = 50 # Load Resistor, Ohms
G = 10 # Post Load Amplifier Gain
Pave = 5/1000; sigma = 5.0 # Divide Pave by 1000 to give voltage in mV
bandwidth = 2.5

# Find Q and BEF
for_std1=np.zeros(1000)
for i in range(1000):
    v1 = np.random.normal(np.random.randint(0, 2)*Pave*2*R*RL*G, sigma, 1000)
    v2 = np.random.normal(np.random.randint(0, 2)*Pave*2*R*RL*G, sigma, 1000)
    v3 = np.random.normal(np.random.randint(0, 2)*Pave*2*R*RL*G, sigma, 1000)
    v = np.concatenate([v1,v2,v3])
    tj = t - np.random.normal(0,20)
    
    sig_fft= np.fft.rfft(v)
    fft_freq = np.linspace(0,1500/1.200,1501)
    filter=1/(fft_freq**2/bandwidth**2+1)+1e-2
    sig_fft_f = sig_fft*filter
    vf=np.fft.irfft(sig_fft_f)
    for_std1[i]=vf[1500]
Q = 2*np.mean(for_std1)/(2*np.std(np.absolute(for_std1-1.25)))
BER =  0.5*special.erfc(Q/np.sqrt(2))

v1 = np.random.normal(0, sigma, 1000)
v2 = np.random.normal(Pave*2*R*RL*G, sigma, 1000)
v3 = np.random.normal(0, sigma, 1000)
v = np.concatenate([v1,v2,v3])
t=np.linspace(0,1200,3000) # time in picoseconds
zeros=np.zeros(3000)
Threshold3=zeros+Pave*R*RL*G

sig_fft= np.fft.rfft(v)
fft_freq = np.linspace(0,1500/1.200,1501)
filter=1/(fft_freq**2/bandwidth**2+1)+1e-2
sig_fft_f = sig_fft*filter
vf=np.fft.irfft(sig_fft_f)

p4 = figure(plot_width=400, plot_height=400, background_fill_color="lightgray", title='BER='+'{:.2e}'.format(BER))
p4.title.text_font_style='normal'; p4.title.text_font_size='12pt'
p4.outline_line_width = 1; p4.outline_line_color = "black"; p4.min_border_top = 10
p3.x_range = Range1d(0, 1200); p3.y_range = Range1d(-0.75, 3.25)
eye0=p4.line(t,zeros-2,line_color='#2222aa')
eye1=p4.line(t,zeros-2,line_color='#2222aa')
eye2=p4.line(t,zeros-2,line_color='#2222aa')
eye3=p4.line(t,zeros-2,line_color='#2222aa')
eye4=p4.line(t,zeros-2,line_color='#2222aa')
eye5=p4.line(t,zeros-2,line_color='#2222aa')
eye6=p4.line(t,zeros-2,line_color='#2222aa')
eye7=p4.line(t,zeros-2,line_color='#2222aa')
eye8=p4.line(t,zeros-2,line_color='#2222aa')
eye9=p4.line(t,zeros-2,line_color='#2222aa')
eye10=p4.line(t,zeros-2,line_color='#2222aa')
eye11=p4.line(t,zeros-2,line_color='#2222aa')
eye12=p4.line(t,zeros-2,line_color='#2222aa')
eye13=p4.line(t,zeros-2,line_color='#2222aa')
eye14=p4.line(t,zeros-2,line_color='#2222aa')
eye15=p4.line(t,zeros-2,line_color='#2222aa')
eye16=p4.line(t,zeros-2,line_color='#2222aa')
eye17=p4.line(t,zeros-2,line_color='#2222aa')
eye18=p4.line(t,zeros-2,line_color='#2222aa')
eye19=p4.line(t,zeros-2,line_color='#2222aa')
eye20=p4.line(t,zeros-2,line_color='#2222aa')
p4.line(t,Threshold3,line_dash='dashed',line_color='red')
p4.x_range = Range1d(200, 1000); p4.y_range = Range1d(-0.5, 3.0)
p4.xaxis.axis_label="Time (psec)"; p4.xaxis.major_label_text_font_size = "12pt"
p4.xaxis.axis_label_text_font_style = "normal"; p4.xaxis.axis_label_text_font_size = "12pt"
p4.yaxis.axis_label="Voltage (mV)"; p4.yaxis.major_label_text_font_size = "12pt"
p4.yaxis.axis_label_text_font_style = "normal"; p4.yaxis.axis_label_text_font_size = "12pt";
label5=Label(x=800, y=5.5, text='Q='+str(Q), text_color='darkblue', background_fill_color='lightgray')
label6=Label(x=475, y=Pave*R*RL*G, text='V Threshold', text_color='darkblue', background_fill_color='lightgray')
p4.add_layout(label5); p4.add_layout(label6)

count = 0
for_std=np.zeros(21)
for_std[0]=vf[1500]

def update_plot(v):
    global count
    global for_std

    v1 = np.random.normal(np.random.randint(0, 2)*Pave*2*R*RL*G, sigma, 1000)
    v2 = np.random.normal(np.random.randint(0, 2)*Pave*2*R*RL*G, sigma, 1000)
    v3 = np.random.normal(np.random.randint(0, 2)*Pave*2*R*RL*G, sigma, 1000)
    v = np.concatenate([v1,v2,v3])
    tj = t - np.random.normal(0,20)
    
    sig_fft= np.fft.rfft(v)
    fft_freq = np.linspace(0,1500/1.200,1501)
    filter=1/(fft_freq**2/bandwidth**2+1)+1e-2
    sig_fft_f = sig_fft*filter
    vf=np.fft.irfft(sig_fft_f)

    if count == 0:
        eye0.visible=True
        eye0.data_source.data['x']=tj
        eye0.data_source.data['y']=vf
        for_std[count]=vf[1500]
    if count == 1:
        eye1.visible=True
        eye1.data_source.data['x']=tj
        eye1.data_source.data['y']=vf
        for_std[count]=vf[1500]
    elif count == 2:
        eye2.visible=True
        eye2.data_source.data['x']=tj
        eye2.data_source.data['y']=vf
        for_std[count]=vf[1500]
    elif count == 3:
        eye3.visible=True
        eye3.data_source.data['x']=tj
        eye3.data_source.data['y']=vf
        for_std[count]=vf[1500]
    elif count == 4:
        eye4.visible=True      
        eye4.data_source.data['x']=tj
        eye4.data_source.data['y']=vf
        for_std[count]=vf[1500]
    elif count == 5:
        eye5.visible=True
        eye5.data_source.data['x']=tj
        eye5.data_source.data['y']=vf
        for_std[count]=vf[1500]
    elif count == 6:
        eye6.visible=True
        eye6.data_source.data['x']=tj
        eye6.data_source.data['y']=vf
        for_std[count]=vf[1500]
    elif count == 7:
        eye7.visible=True
        eye7.data_source.data['x']=tj
        eye7.data_source.data['y']=vf
        for_std[count]=vf[1500]
    elif count == 8:
        eye8.visible=True
        eye8.data_source.data['x']=tj
        eye8.data_source.data['y']=vf
        for_std[count]=vf[1500]
    elif count == 9:
        eye9.visible=True
        eye9.data_source.data['x']=tj
        eye9.data_source.data['y']=vf
        for_std[count]=vf[1500]
    elif count == 10:
        eye10.visible=True
        eye10.data_source.data['x']=tj
        eye10.data_source.data['y']=vf
        for_std[count]=vf[1500]
    elif count == 11:
        eye11.visible=True
        eye11.data_source.data['x']=tj
        eye11.data_source.data['y']=vf
        for_std[count]=vf[1500]
    elif count == 12:
        eye12.visible=True
        eye12.data_source.data['x']=tj
        eye12.data_source.data['y']=vf
        for_std[count]=vf[1500]
    elif count == 13:
        eye13.visible=True
        eye13.data_source.data['x']=tj
        eye13.data_source.data['y']=vf
        for_std[count]=vf[1500]
    elif count == 14:
        eye14.visible=True
        eye14.data_source.data['x']=tj
        eye14.data_source.data['y']=vf
        for_std[count]=vf[1500]
    elif count == 15:
        eye15.visible=True
        eye15.data_source.data['x']=tj
        eye15.data_source.data['y']=vf
        for_std[count]=vf[1500]
    elif count == 16:
        eye16.visible=True
        eye16.data_source.data['x']=tj
        eye16.data_source.data['y']=vf
        for_std[count]=vf[1500]
    elif count == 17:
        eye17.visible=True
        eye17.data_source.data['x']=tj
        eye17.data_source.data['y']=vf
        for_std[count]=vf[1500]
    elif count == 18:
        eye18.visible=True
        eye18.data_source.data['x']=tj
        eye18.data_source.data['y']=vf
        for_std[count]=vf[1500]
    elif count == 19:
        eye19.visible=True
        eye19.data_source.data['x']=tj
        eye19.data_source.data['y']=vf
        for_std[count]=vf[1500]
    elif count == 20:
        eye20.visible=True
        eye20.data_source.data['x']=tj
        eye20.data_source.data['y']=vf
        count = -1
    count += 1
    push_notebook(handle=fig_handle3)
    
fig_handle3 = show(p4, notebook_handle=True)
trace_button.on_click(update_plot)

def initialize_plot(pa):
    global count
    eye0.visible=False; eye1.visible=False; eye2.visible=False; eye3.visible=False; eye4.visible=False
    eye5.visible=False; eye6.visible=False; eye7.visible=False; eye8.visible=False; eye9.visible=False;
    eye10.visible=False; eye11.visible=False; eye12.visible=False; eye13.visible=False; eye14.visible=False;
    eye15.visible=False; eye16.visible=False; eye17.visible=False; eye18.visible=False; eye19.visible=False;
    eye20.visible=False
    count = 0
    push_notebook(handle=fig_handle3)

reset_button.on_click(initialize_plot)